In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform  # Import uniform
from sklearn.ensemble import RandomForestRegressor  # Import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge  # Import Ridge



In [8]:
df = pd.read_csv('Final_v1.csv')

In [9]:
df = df[['id', 'action_time', 'unsurity', 'structural_change',
       'long_paste', 'unproductive_time', 'external_help',
       'pasted_words_number', 'large_changes','score']]

In [10]:
df.head()

,id,action_time,unsurity,structural_change,long_paste,unproductive_time,external_help,pasted_words_number,large_changes,score
0,001519c8,12.602309,0.000000,0.000000,0.0,9.825904,0.0,0.000000,0.000000,3.5
1,0022f953,12.525951,0.000000,0.000000,0.0,9.531119,0.0,0.000000,0.000000,3.5
2,0042269b,12.950868,0.693147,1.609438,0.0,10.432703,0.0,0.000000,1.791759,6.0
3,0059420b,12.152656,0.000000,0.000000,0.0,8.027150,0.0,0.693147,0.000000,2.0
4,0075873a,12.656202,0.000000,0.000000,0.0,8.852093,0.0,0.000000,0.000000,4.0


In [11]:
X = df.drop(['id', 'score'], axis=1)
y = df['score']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

# Applying PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

X_test = 

# Models to train: RandomForestRegressor, GradientBoostingRegressor, Ridge
models = {
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "Ridge": Ridge()
}

# Parameter distributions for RandomizedSearchCV
param_distributions = {
    "RandomForestRegressor": {'n_estimators': sp_randint(100, 200), 'max_depth': [None, 10, 20]},
    "GradientBoostingRegressor": {'n_estimators': sp_randint(100, 200), 'learning_rate': uniform(0.01, 0.5), 'max_depth': [3, 5, 10]},
    "Ridge": {'alpha': uniform(0.1, 10)}
}

# Running RandomizedSearchCV for each model
best_estimators = {}
for model_name in models:
    random_search = RandomizedSearchCV(estimator=models[model_name], param_distributions=param_distributions[model_name], n_iter=10, cv=5, random_state=0)
    random_search.fit(X_train, y_train)
    best_estimators[model_name] = random_search.best_estimator_

# Evaluating the models
mse_scores = {}
for model_name, model in best_estimators.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores[model_name] = mse

# Displaying the MSE scores
mse_scores

/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


{'RandomForestRegressor': 0.9101702109316404,
 'GradientBoostingRegressor': 1.0115014594626157,
 'Ridge': 1.0326654400078852}

In [12]:
# Evaluating the models
mse_scores = {}
for model_name, model in best_estimators.items():
    y_pred = model.predict(X_test)
    y_pred_rounded = np.round(y_pred)  # Round off predictions
    mse = mean_squared_error(y_test, y_pred_rounded)
    mse_scores[model_name] = mse

# Displaying the MSE scores
mse_scores


{'RandomForestRegressor': 0.9247474747474748,
 'GradientBoostingRegressor': 1.0196969696969698,
 'Ridge': 1.155050505050505}